In [ ]:
local = False #Set to False if working with a web-hosted Jupyter server
#Core libraries
import os
import base64
import re
from time import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#Third party libraries
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Layout
from IPython.display import Javascript, display
from bqplot import *
from bqplot import pyplot as plt
from bqplot.interacts import *
import numpy as np
#Custom libraries
from SelectFilesWidget import *
from tools import *
from def_strain import *
from def_DW import *
from def_XRD import *
from def_fit import *

In [ ]:
#*******************************************************************
#                WIDGET: XRD data selection
#  
#  This allows to load a file from the local drive. 
#*******************************************************************

dirname = ''
filename = ''
#load default data
tth,iexp = np.loadtxt("Example_data.txt", unpack = True) #Read experimental data
iexp[iexp==0] = np.min(iexp[np.nonzero(iexp)]) #Replace any 0s with the minimum value
iexp /= iexp.max() #Normalize to 1
th = tth * np.pi/360 #Convert 2theta angle -> theta angles in radians

#*** !TO BE USED WHEN THE NOTEBOOK IS RUNNED LOCALLY ***
try:
    f=open("last_path", "r")
    init_dir=f.read()
    f.close()
except:
    init_dir=os.getcwd()
load_button = SelectFilesButton() #this is the custom load file button
load_button.description = "NO DATA... PLEASE LOAD XRD FILE." 
load_button.layout.width = "50%"
load_button.default_path = init_dir

#***  !TO BE USED WHEN THE NOTEBOOK IS HOSTED ON THE WEB ***
upload_button=widgets.FileUpload(
    description='Upload XRD data',
    accept='.txt',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False,  # True to accept multiple files upload else False
    layout=Layout(width='25%')
)

In [ ]:
#***************************************************************************
# WIDGETS: define the experiment / material / Strain+Debye-Waller parameters
#***************************************************************************

resol_fct = widgets.Dropdown(
    options=['Pseudo-Voigt', 'Gaussian', 'Lorentzian', 'Generalized bell', 'Split pseudo-Voigt'],
    value='Gaussian',
    description='Resolution',
    disabled=False,
    continuous_update=False
)

resol_width = widgets.Text(
    value='0.01',
    placeholder='Full width at half-maximum',
    description='FWHM (°)',
    disabled=False,
    continuous_update=False,
)
resol_shape = widgets.Text(
    value='0.001',
    placeholder='eta',
    description='Shape',
    disabled=False,
    continuous_update=False
)

# Wavelength, offset and background
exp_wl = widgets.Text(
    value='1.5406',
    placeholder='Wavelength',
    description='Wavelength '+ u"(\u212B)",
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'}
)
exp_offset = widgets.Text(
    value='0.',
    placeholder='2 theta offset',
    description='2'+u"\u03B8"+' offset (°)',
    disabled=False,
    continuous_update=False
)
exp_bkg = widgets.Text(
    value='0.',
    placeholder='Background',
    description='Background',
    disabled=False,
    continuous_update=False
)

#*** Widgets for the "Material" tab ***
# Type of crystal and Miller indices
cryst_name = widgets.Dropdown(
    options=sorted(os.listdir('structures/')),
    value='ZrO2.cif',
    description='Material',
    disabled=False,
)
cryst_h = widgets.Text(
    value='0',
    placeholder='h',
    description='h',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='20%')
)
cryst_k = widgets.Text(
    value='0',
    placeholder='k',
    description='k',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='20%')
)
cryst_l = widgets.Text(
    value='4',
    placeholder='l',
    description='l',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='20%')
)

# Sample type (single crystal or film)
sample = widgets.Dropdown(
    options=['Single crystal', 'Thin film', 'Thick film', 'Thick film + substrate'],
    value='Single crystal',
    description='Sample type',
    disabled=False,
)

#*** Widgets for the "Strain / Disorder" tab ***
# Spline model for strain and DW depth profiles
sdw_model = widgets.Dropdown(
    options=['B-splines smooth', 'B-splines abrupt'],
    value='B-splines smooth',
    description='Strain/DW depth-profile',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='29%')
)
sdw_basis = widgets.Text(
    value='6',
    placeholder='Number of basis functions',
    description='Control points',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='15%')
)

#Strain V-Scale sliders
strain_scale = widgets.FloatSlider(
    value=1,
    min=0.5, 
    max=1.5, 
    step=0.01,
    description='Strain: scale',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True
)
#Strain H-Scale sliders
strain_shift = widgets.FloatSlider(
    value=0,
    min=-0.1, 
    max=0.1, 
    step=0.01,
    description='shift',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True
)

# Irradiated thickness and number of slices used for the XRD computation
th_value = widgets.Text(
    value='100',
    placeholder='Damaged depth (nm)',
    description='Damaged depth (nm)    ',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='29%')
)
th_slices = widgets.Text(
    value='60',
    placeholder='Number of sub-layers',
    description='Data points',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='15%')
)
#DW V-Scale sliders
dw_scale = widgets.FloatSlider(
    value=1,
    min=0.5,
    max=1.5,
    step=0.01,
    description='DW: scale',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True
)
#DW H-Scale sliders
dw_shift = widgets.FloatSlider(
    value=0,
    min=-0.1,
    max=0.1,
    step=0.01,
    description='shift',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout = True
)

#*** Widgets for the "Film parameters" tab ***
text_film = widgets.HTML(
    value='Modify these parameters only if you are using the "<u>thick film</u>" or the "<u>thick film+substrate</u>" sample types.'
)
fth_value = widgets.Text(
    value='1000',
    description='Film thickness (nm)',
    placeholder='Film thickness has to be larger than damaged depth',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='20%')
)    

sub_name = widgets.Dropdown(
    options=sorted(os.listdir('structures/')),
    value=sorted(os.listdir('structures/'))[-1],
    description='Substrate',
    disabled=False,
    layout=Layout(width='20%')
)
sub_h = widgets.Text(
    value='0',
    placeholder='h',
    description='h',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='20%')
)
sub_k = widgets.Text(
    value='0',
    placeholder='k',
    description='k',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='19%')
)
sub_l = widgets.Text(
    value='4',
    placeholder='l',
    description='l',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='19%')
)
#*** Widgets for the "Fitting parameters" tab ***
algo = widgets.Dropdown(
    options=['Least squares', 'Least squares (no bounds)', 'Simulated annealing (GSA)'],
    value='Least squares',
    description='Fitting algorithm',
    disabled=False,
    style = {'description_width': 'initial'},
    layout=Layout(width='25%')
)
min_strain = widgets.Text(
    value='0',
    description='Min. strain',
    placeholder='Lower limit for the strain',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='19%')
)
max_strain = widgets.Text(
    value='5',
    description='Max. strain',
    placeholder='Upper limit for the strain',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='18%')
)
min_dw = widgets.Text(
    value='0',
    description='Min. DW',
    placeholder='Lower limit for the Debye-Waller',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='18%')
)
max_dw = widgets.Text(
    value='1',
    description='Max. DW',
    placeholder='Upper limit for the Debye-Waller',
    disabled=False,
    continuous_update=False,
    layout=Layout(width='18%')
)
text_gsa = widgets.HTML(
    value = 'GSA parameters:',
    layout=Layout(width='15%')
)
gsa_temp = widgets.Text(
    value='100',
    description='Temperature',
    placeholder='Simulated annealing pseudo-temperature [1-1000]',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'}   
)
gsa_cycles = widgets.Text(
    value='1000',
    description='Monte-Carlo steps',
    placeholder='Number of Monte Carlo cycles in the GSA routine',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'}
)
gsa_Tsteps = widgets.Text(
    value='10',
    description='Cooling stages',
    placeholder='How many temperature dwells',
    disabled=False,
    continuous_update=False,
    style = {'description_width': 'initial'}
)

w_list = [resol_fct,     #0
          resol_width,   #1
          resol_shape,   #2
          exp_wl,        #3
          exp_offset,    #4
          exp_bkg,       #5
          cryst_name,    #6
          cryst_h,       #7
          cryst_k,       #8
          cryst_l,       #9
          sample,        #10
          sdw_model,     #11
          sdw_basis,     #12
          th_value,      #13
          th_slices,     #14
          fth_value,     #15
          sub_name,      #16
          sub_h,         #17
          sub_k,         #18
          sub_l,         #19
          algo,          #20
          min_strain,    #21
          max_strain,    #22
          min_dw,        #23
          max_dw,        #24
          gsa_temp,      #25
          gsa_cycles,    #26
          gsa_Tsteps]    #27

# Compute global variables from the widgets
cst = compute_cst(w_list,th)

In [ ]:
#*******************************************************************
#       WIDGETS: interactive plots of strain, DW and XRD 
#*******************************************************************

eps = max(auto_strain(tth, iexp, cst), 0.1)
sp = np.full(cst["sdw_basis"], eps)
dwp = np.ones(cst["sdw_basis"])        
        
#Create backup (used for to restore saved data when fit is cancelled)     
sp_back, dwp_back = np.copy(sp), np.copy(dwp)

# compute the initial strain, DW and intensity
strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
error = rmse(iexp,ical)

old_strain = np.copy(strain)
old_dw = np.copy(dw)

# compute the values of strain and DW at the control point coords (in1d, around)
control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])

#********** Strain figure **********
xscale = LinearScale()
yscale = LinearScale()
xax = Axis(scale=xscale, label='', grids='off')
yax = Axis(scale=yscale, label='Strain (%)', orientation='vertical', grids='off')
strain_line = Lines(x=(cst["t"]-cst["z"])/10,
                    y=strain*100,
                    scales={'x': xscale, 'y': yscale})
strain_scat = Scatter(x=(cst["t"] - control_sp_x)/10, 
                      y=control_sp_y*100,
                      interactions={"click": "select"},
                      selected_style={"opacity": 1.0, "fill": "DarkOrange", "stroke": "Red"},
                      enable_move=True, restrict_y = True,default_size=128,
                      scales={'x': xscale, 'y': yscale})
fig_strain=Figure(marks=[strain_scat, strain_line], axes=[xax, yax], animation_duration = 300)

#********** DW figure **********
xscale = LinearScale()
yscale = LinearScale(min=0)#, max = 1.1) #MODIFY HERE TO REMOVE DW UPPER BOUND IN PLOT
xax = Axis(scale=xscale, label='Depth (nm)')
yax = Axis(scale=yscale, label='Debye-Waller factor', orientation='vertical', grids='off')
dw_line = Lines(x=(cst["t"]-cst["z"])/10,
                    y=dw,
                    scales={'x': xscale, 'y': yscale})
dw_scat = Scatter(x=(cst["t"] - control_dwp_x)/10, 
                      y=control_dwp_y,
                      interactions={"click": "select"},
                      selected_style={"opacity": 1.0, "fill": "DarkOrange", "stroke": "Red"},
                      enable_move=True, restrict_y = True,default_size=128,
                      scales={'x': xscale, 'y': yscale})
fig_DW=Figure(marks=[dw_scat, dw_line], axes=[xax, yax], animation_duration = 300)

#********** XRD figure **********
xscale = LinearScale()
yscale = LinearScale()
xrd_scat = Scatter(x=th*360/np.pi, y=np.log10(iexp), scales={'x': xscale, 'y': yscale})
xrd_line = Lines(x=th*360/np.pi, y=np.log10(ical), scales={'x': xscale, 'y': yscale}, colors=['red'])

panzoom = PanZoom(scales={'x': [xscale], 'y': [yscale]})
xax = Axis(scale=xscale, label='2'+u"\u03B8"+' (deg.)', grids='off')
yax = Axis(scale=yscale, label='', orientation='vertical', grid_lines='none', visible=False)

fig_XRD = Figure(marks=[xrd_scat, xrd_line], axes=[xax, yax], animation_duration = 300,interaction=panzoom)

In [ ]:
#*******************************************************************
#              WIDGETS: SAVE EVERYTHING / FIT / CANCEL FIT
#*******************************************************************

save = widgets.Button(
    description='Save',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Save all parameters, depth-profiles and simulation'
)

fit = widgets.Button(
    description='Fit',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Fit experimental data'
)

cancel = widgets.Button(
    description='Cancel',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Reload previous strain/DW values'
)

out = widgets.Output() #This is an output text area below the GUI
with out:
    print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))

In [ ]:
#*******************************************************************
#    EVENT LOOP: OBSERVE ALL WIDGET VALUES AND MOUSE EVENTS
#*******************************************************************
#Observe the file load widget (local version)
def on_file_change(change):
    global dirname, filename, th, iexp, cst, sp, dwp, old_strain, old_dw
    #open data file
    dirname = os.path.dirname(load_button.files[0])
    filename = os.path.basename(load_button.files[0])
    load_button.description = "File: "+ os.path.join(dirname, filename)
    
    #save data file path
    f = open("last_path", "w")
    f.write(dirname)
    f.close()
    
    #load the xrd data
    tth,iexp = np.loadtxt(load_button.files[0], unpack = True) #Read experimental data
    iexp[iexp==0] = np.min(iexp[np.nonzero(iexp)]) #Replace any 0s with the minimum value
    iexp /= iexp.max() #Normalize to 1
    th = tth * np.pi/360 #Convert 2theta angle -> theta angles in radians
    
    #check if there are widget values stored in the folder. If none keep defaults
    try:
        ipyw_values = np.load(os.path.join(dirname, "ipyw_values.npy"), allow_pickle=True).item()
        ipyw_flag = 1
        with out:
            out.clear_output()
            print("Found saved session")
    except:
        ipyw_flag = 0
        with out:
            out.clear_output()
            print("No saved session found")
    if ipyw_flag == 1:
        for i,key in enumerate(ipyw_values):
            w_list[i].value = ipyw_values[key]

    #Check if there are saved B-spline weights. If none keep defaults
    try:
        sp, dwp = np.loadtxt(os.path.join(dirname, "weights.txt"), unpack=True)
        #Create backup (used for to restore saved data when fit is cancelled)     
        sp_back, dwp_back = np.copy(sp), np.copy(dwp)
        #compute the initial strain, DW and intensity
        strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
        dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
        control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
        control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
        with out:
            print("Found saved B-spline weights")
    except:
        eps = max(auto_strain(tth, iexp, cst), 0.1)
        sp = np.full(cst["sdw_basis"], eps)
        dwp = np.ones(cst["sdw_basis"]) 
        sp_back, dwp_back = np.copy(sp), np.copy(dwp)
        strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
        dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
        control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
        control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
        with out:
            print("No B-spline weights found")
    
    #update the strain/DW plots    
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y
    
    #compute xrd
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    with out:
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    
    #update xrd plot
    with xrd_scat.hold_sync():
        xrd_scat.x = th*360/np.pi
        xrd_scat.y = np.log10(iexp)
    with xrd_line.hold_sync():
        xrd_line.x = th*360/np.pi
        xrd_line.y = np.log10(ical)
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)
       
load_button.observe(on_file_change, names='files')


#Observe the file load widget (online version)
@out.capture(clear_output=True)
def on_file_upload(change):
    global dirname, filename, th, iexp, cst, sp, dwp, old_strain, old_dw
        
    filename = (upload_button.metadata[0]['name']) #get the file name from the embedded metadata dict
    raw_data = upload_button.value[filename]['content'].decode() #extract data byte string and convert to str
    try:
        data = np.fromstring(raw_data, dtype=float, sep = ' ') #convert str to 1D np array
        data = data.reshape(int(len(data)/2),2) #reshape to 2cols format
        tth,iexp = data[:,0], data[:,1] #Read experimental data
        iexp[iexp==0] = np.min(iexp[np.nonzero(iexp)]) #Replace any 0s with the minimum value
        iexp /= iexp.max() #Normalize to 1
        th = tth * np.pi/360 #Convert 2theta angle -> theta angles in radians
        # guess strain from data and generate profiles
        eps = max(auto_strain(tth, iexp, cst), 0.1)
        sp = np.full(cst["sdw_basis"], eps)
        dwp = np.ones(cst["sdw_basis"])
        strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
        dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
        # generate control points
        control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
        control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
        #compute XRD
        ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
        error = rmse(iexp,ical)
        with out:
            print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    except:
        with out:
            print("ERROR. CHECK INPUT DATA: Numbers only in 2-columns space-separated values")
            print("PLEASE REFRESH PAGE.")
    
    
    
    #update xrd plot
    #panzoom = PanZoom(scales={'x': [xscale], 'y': [yscale]})
    xscale.min = th.min()*360/np.pi
    xscale.max = th.max()*360/np.pi
    yscale.min = np.log10(iexp.min())
    yscale.max = np.log10(iexp.max())
    with xrd_scat.hold_sync():
        xrd_scat.x = th*360/np.pi
        xrd_scat.y = np.log10(iexp)
    with xrd_line.hold_sync():
        xrd_line.x = th*360/np.pi
        xrd_line.y = np.log10(ical)
    #update strain/DW plot    
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y
    
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)
    
            
upload_button.observe(on_file_upload, names='value')


#observe input widgets and update changes
@out.capture(clear_output=True)
def update_xrd(change):
    #update the XRD curve only
    global cst
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    xrd_line.y = np.log10(ical)
    error = rmse(iexp,ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))

@out.capture(clear_output=True)        
def update_all(change):
    #update the strain/DW and XRD curves upon widget modification (except Nb of Bsplines)
    global cst, control_sp_x, control_sp_y, control_dwp_x, control_dwp_y, old_strain, old_dw
    cst = compute_cst(w_list,th)
    # update strain and DW
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the values of strain and DW at the control point coords (in1d, around)
    control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
    control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the plots
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y
    # update XRD
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    xrd_line.y = np.log10(ical)
    error = rmse(iexp,ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)

@out.capture(clear_output=True)
def update_all_w_basis(change):
    #update the strain/DW and XRD curves upon Nb of Bsplines modification
    global cst, sp, dwp, control_sp_x, control_sp_y, control_dwp_x, control_dwp_y, old_strain, old_dw
    #read the new value for the number of basis functions
    cst = compute_cst(w_list,th)
    #compute the new weights
    sp_new = old2new_strain(cst["z"], sp, cst["t"], cst["sdw_basis"],cst["sdw_model"])
    dwp_new = old2new_DW(cst["z"], dwp, cst["t"], cst["sdw_basis"],cst["sdw_model"])
    sp, dwp = sp_new, dwp_new
    #compute the corresponding strain and DW
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the values of strain and DW at the control point coords (in1d, around)
    control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
    control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the plots
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y
    # update XRD
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    xrd_line.y = np.log10(ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
        
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)

#observe all widgets
for i, widget in enumerate(w_list):
    if i<=10:
        widget.observe(update_xrd, names='value')
    elif i==12:
        widget.observe(update_all_w_basis, names='value')
    else:
        widget.observe(update_all, names='value')

#observe the strain slider      
@out.capture(clear_output=True)
def update_strain_scale(change=None):
    global sp, strain, control_sp_y, cst, old_strain
    #get scale factor from widget and modify control points accordingly
    control_sp_y = control_sp_y*strain_scale.value
    strain_scat.y = control_sp_y*100
    #update strain curve
    sp = interp_and_fit_strain(control_sp_x, control_sp_y, cst["z"], sp, cst["sdw_model"]) #compute the weights
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    strain_line.y = strain*100 #redraw new strain
    #switch slider back to default value = 1
    strain_scale.value = 1
    #update XRD curve
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    xrd_line.y = np.log10(ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)

strain_scale.observe(update_strain_scale, names = 'value')

#observe the strain shift slider      
@out.capture(clear_output=True)
def update_strain_shift(change=None):
    global sp, strain, control_sp_y, cst, old_strain
    # create an array twice the initial size to be shifted
    length = int(len(old_strain))
    expanded_strain = np.zeros(2*length)
    expanded_strain[int(length/2):int(3*length/2)]=old_strain
    expanded_strain[int(3*length/2):2*length]=old_strain[-1]
    # get shift factor from widget and shift curve accordingly    
    cut = int(strain_shift.value * length)
    shifted_strain = expanded_strain[int(length/2)+cut:int(3*length/2)+cut]
    
    # compute new weights, new strain and update strain curve
    sp = shift_strain(cst["z"], sp, cst["t"], shifted_strain, cst["sdw_model"])
    control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
    strain = f_strain(cst["z"], sp, cst["t"], cst["sdw_model"])
    # update plot (for some reason there is no need to update the XRD. Its done via observe_strain_plot)
    strain_line.y = strain*100 #redraw new strain
    strain_scat.y = control_sp_y*100
    # switch slider back to default value = 0
    strain_shift.value = 0
strain_shift.observe(update_strain_shift, names = 'value')

#observe the dw slider 
@out.capture(clear_output=True)
def update_dw_scale(change=None):
    global dwp, dw, control_dwp_y, cst, old_dw
    #get scale factor from widget and modify control points accordingly
    control_dwp_y = control_dwp_y*dw_scale.value
#     control_dwp_y[control_dwp_y>1] = 1  #TEMPORARILY REMOVE DW UPPER LIMIT
    dw_scat.y = control_dwp_y
    #update strain curve
    dwp = interp_and_fit_dw(control_dwp_x, control_dwp_y, cst["z"], dwp, cst["sdw_model"]) #compute the weights
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    dw_line.y = dw #redraw new strain
    #switch slider back to default value = 1
    dw_scale.value = 1.
    #update XRD and update the curve
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    xrd_line.y = np.log10(ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    # create a strain/dw backup (for shift)
    old_dw = np.copy(dw)
dw_scale.observe(update_dw_scale, names = 'value')

#observe the dw shift slider      
@out.capture(clear_output=True)
def update_dw_shift(change=None):
    global dwp, dw, control_dwp_y, cst, old_dw
    # create an array twice the initial size to be shifted
    length = int(len(old_dw))
    expanded_dw = np.ones(2*length)
    expanded_dw[int(length/2):int(3*length/2)]=old_dw
    expanded_dw[int(3*length/2):2*length]=old_dw[-1]
    # get shift factor from widget and shift curve accordingly    
    cut = int(dw_shift.value * length)
    shifted_dw = expanded_dw[int(length/2)+cut:int(3*length/2)+cut]
    
    # compute new weights, new strain and update strain curve
    dwp = shift_dw(cst["z"], dwp, cst["t"], shifted_dw, cst["sdw_model"])
    control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"], dwp, cst["t"], cst["sdw_model"])
    # update plot (for some reason there is no need to update the XRD. Its done via observe_strain_plot)
    dw_line.y = dw #redraw new strain
    dw_scat.y = control_dwp_y
    # switch slider back to default value = 0
    dw_shift.value = 0
dw_shift.observe(update_dw_shift, names = 'value')

#observe the strain interactive plot
@out.capture(clear_output=True)
def update_strain_xrd(change=None):
    global sp, control_sp_x, control_sp_y, cst, old_strain
    
    # get the new control point values
    if np.any(strain_scat.selected):
        old_control_sp_y = np.copy(control_sp_y)
        if len(strain_scat.selected)>1:
            with out:
                print("Moving multiple points:", *strain_scat.selected)
        delta_strain = change['new']/100 - old_control_sp_y #get old-new difference array
        delta_strain = delta_strain[np.nonzero(delta_strain)] #find the delta value
        new_control_sp_y = old_control_sp_y
        new_control_sp_y[strain_scat.selected] += delta_strain #shift only selected points
    else:        
        new_control_sp_y = change['new']/100 #get new strain values
        
    #compute the new weights and update the curve
    sp = interp_and_fit_strain(control_sp_x, new_control_sp_y, cst["z"], sp, cst["sdw_model"]) 
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100 #redraw new strain
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = new_control_sp_y*100    
        
    control_sp_y = new_control_sp_y #update the control point values
    #update XRD
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    xrd_line.y = np.log10(ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)

strain_scat.observe(update_strain_xrd, names=['y'])

#observe the dw interactive plot
@out.capture(clear_output=True)
def update_dw_xrd(change=None):
    global dwp, control_dwp_x, control_dwp_y, cst, old_dw
    
    # get the new control point values
    if np.any(dw_scat.selected):
        old_control_dwp_y = np.copy(control_dwp_y)
        if len(dw_scat.selected)>1:
            with out:
                print("Moving multiple points:", *dw_scat.selected)
        delta_dw = change['new'] - old_control_dwp_y #get old-new difference array
        delta_dw = delta_dw[np.nonzero(delta_dw)] #find the delta value
        new_control_dwp_y = old_control_dwp_y
        new_control_dwp_y[dw_scat.selected] += delta_dw #shift only selected points
    else:        
        new_control_dwp_y = change['new'] #get new strain values

    #compute the weights 
    dwp = interp_and_fit_dw(control_dwp_x, new_control_dwp_y, cst["z"], dwp, cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw #redraw new dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = new_control_dwp_y
        
    control_dwp_y = new_control_dwp_y #update the control point values
    #update XRD and update the curve
    cst = compute_cst(w_list,th)
    ical, t0 = f_Refl(th, np.concatenate((sp,dwp)), cst)
    error = rmse(iexp,ical)
    xrd_line.y = np.log10(ical)
    with out:
        out.clear_output()
        print("Computing time: %f4 sec.   RMS error: %f4"%(t0,error))
    # create a strain/dw backup (for shift)
    old_dw = np.copy(dw)

dw_scat.observe(update_dw_xrd, names=['y'])

#observe the save button
def on_save_clicked(b):
    #save the widget values
    ipyw_values = {
    "resol_fct": resol_fct.value,
    "resol_width": resol_width.value,
    "resol_shape": resol_shape.value,
    "exp_wl": exp_wl.value,
    "exp_offset": exp_offset.value,
    "exp_bkg": exp_bkg.value,
    "cryst_name": cryst_name.value,
    "cryst_h": cryst_h.value,
    "cryst_k": cryst_k.value,
    "cryst_l": cryst_l.value,
    "sample": sample.value,
    "sdw_model": sdw_model.value,
    "sdw_basis": sdw_basis.value,
    "th_value": th_value.value,
    "th_slices": th_slices.value,
    "fth_value": fth_value.value,
    "sub_name": sub_name.value,
    "sub_h": sub_h.value,
    "sub_k": sub_k.value,
    "sub_l": sub_l.value,
    "algo": algo.value,
    "min_strain": min_strain.value,
    "max_strain": max_strain.value,
    "min_dw": min_dw.value,
    "max_dw": max_dw.value,
    "gsa_temp": gsa_temp.value,
    "gsa_cycles": gsa_cycles.value,
    "gsa_Tsteps": gsa_Tsteps.value
    }
    #if the notebook is runned locally, save the widget values to disk
    if local:
        np.save(os.path.join(dirname, "ipyw_values.npy"), ipyw_values)
        with out:
            out.clear_output()
            print("Data saved.")
    
    #save the B-spline weights. If the notebook is runned locally, save the data to disk
    if local: 
        np.savetxt(os.path.join(dirname, "weights.txt"), np.column_stack((sp,dwp)), fmt="%10.8f")
    
    #save the XRD simulation
    #if the notebook is runned locally, save the data to disk
    #otherwise, the data is downloaded via a base64-encoded data URL
    ical = f_Refl(th, np.concatenate((sp,dwp)), cst)[0]
    out_xrd = np.column_stack((th*360/np.pi,iexp,ical))
    if local: 
        np.savetxt(os.path.join(dirname, "simul_xrd.txt"), out_xrd, fmt="%10.8f")
    else: 
        with out:
            out.clear_output()
        out_xrd = np.array2string(out_xrd, threshold = 1e6) #convert the array to a str
        out_xrd = re.sub('[\[\]]', ' ', out_xrd) #remove the [] brackets
        out_xrd = out_xrd.replace("\n", "%0A") #thanks raphj (@linuxfr.org) for this hack
        #The next 2 lines are needed for base64 encoding only
        #out_xrd = base64.b64encode(out_xrd.encode('ascii')) #convert to base64
        #out_xrd = str(out_xrd).replace("'","").replace("b","") #remove the b and the quotes from b64 string
        xrd_link = """<a download='simul_xrd.txt' href='data:text/csv;charset=utf-8;ascii,"""+out_xrd+"""'>Download XRD simulation.</a>"""
        dl_xrd.value = xrd_link
    
    #save the strain/DW depth-profiles
    #if the notebook is runned locally, save the data to disk
    #otherwise, the data is downloaded via a base64-encoded data URL
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    depth = (cst["t"]-cst["z"])/10
    out_sdw = np.column_stack((depth,strain,dw))
    if local:
        np.savetxt(os.path.join(dirname, "simul_strain_dw.txt"),out_sdw , fmt="%10.8f")
    else:    
        out_sdw = np.array2string(out_sdw, threshold = 1e6) #convert the array to a str
        out_sdw = re.sub('[\[\]]', ' ', out_sdw) #remove the [] brackets
        out_sdw = out_sdw.replace("\n", "%0A") #thanks raphj (@linuxfr.org) for this hack
        #The next 2 lines are needed for base64 encoding only
        #out_sdw = base64.b64encode(out_sdw.encode('ascii')) #convert to base64
        #out_sdw = str(out_sdw).replace("'","").replace("b","") #remove the b and the quotes from b64 string
        sdw_link = "<a download='simul_strain_dw.txt' href='data:text/csv;charset=utf-8;ascii,"+out_sdw+"'>Download strain/DW depth profiles.</a>"
        dl_sdw.value = sdw_link

save.on_click(on_save_clicked)

#observe the fit button
@out.capture(clear_output=True)
def on_fit_clicked(b):
    global cst, sp, sp_back, dwp_back, dwp, control_sp_x, control_sp_y, control_dwp_x, control_dwp_y, old_strain, old_dw
    dl_xrd.value = ""
    dl_sdw.value = ""
    sp_back, dwp_back = np.copy(sp), np.copy(dwp)
    cst = compute_cst(w_list,th)
    with out:
        out.clear_output()
        print ("Fitting... Please Wait.")
    t0 = time()
    sp, dwp = fit_curve(th, iexp, np.concatenate((sp,dwp)), cst)
    fit_time = time() - t0
    
    # update XRD
    ical = f_Refl(th, np.concatenate((sp,dwp)), cst)[0]
    error = rmse(iexp, ical)
    xrd_line.y = np.log10(ical)
           
    # update strain and DW
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the values of strain and DW at the control point coords (in1d, around)
    control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
    control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the plots
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y      
    with out:
        out.clear_output()
        print("Done. Fitting time: %5.4f sec.   RMS error: %f4" %(fit_time, error))
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)

fit.on_click(on_fit_clicked)

#observe the cancel button
def on_cancel_clicked(b):
    global cst, sp, dwp, control_sp_x, control_sp_y, control_dwp_x, control_dwp_y, old_strain, old_dw
    dl_xrd.value = ""
    dl_sdw.value = ""
    cst = compute_cst(w_list,th)
    sp, dwp = np.copy(sp_back), np.copy(dwp_back)
    with out:
        print("Fit cancelled.")
    
    # update strain and DW
    strain = f_strain(cst["z"],sp,cst["t"],cst["sdw_model"])
    dw = f_DW(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the values of strain and DW at the control point coords (in1d, around)
    control_sp_x, control_sp_y = control_sp(cst["z"],sp,cst["t"],cst["sdw_model"])
    control_dwp_x, control_dwp_y = control_dwp(cst["z"],dwp,cst["t"],cst["sdw_model"])
    # update the plots
    with strain_line.hold_sync():
        strain_line.x = (cst["t"]-cst["z"])/10
        strain_line.y = strain*100
    with strain_scat.hold_sync():
        strain_scat.x = (cst["t"] - control_sp_x)/10
        strain_scat.y = control_sp_y*100
    with dw_line.hold_sync():
        dw_line.x = (cst["t"]-cst["z"])/10
        dw_line.y = dw
    with dw_scat.hold_sync():
        dw_scat.x = (cst["t"] - control_dwp_x)/10
        dw_scat.y = control_dwp_y
    # update XRD
    ical = f_Refl(th, np.concatenate((sp,dwp)), cst)[0]
    xrd_line.y = np.log10(ical)
    # create a strain/dw backup (for shift)
    old_strain = np.copy(strain)
    old_dw = np.copy(dw)


cancel.on_click(on_cancel_clicked)

In [ ]:
#*******************************************************************
#                       Generate the GUI
#*******************************************************************
#Experiment tab
resol_box = HBox([resol_fct, resol_width, resol_shape])
exp_box = HBox([exp_wl, exp_offset, exp_bkg])
exp_tab = VBox([resol_box, exp_box])

#Material tab
cryst_box = HBox([cryst_name, cryst_h, cryst_k, cryst_l])

#sample_box = HBox([cryst, cryst_h, cryst_k, cryst_l])
mater_tab = VBox([cryst_box, sample])

#Strain/DW tab
sdw_box = HBox([sdw_model, sdw_basis, strain_scale, strain_shift])
th_box = HBox([th_value, th_slices, dw_scale, dw_shift])
sdw_tab = VBox([sdw_box, th_box])

#Film parameters tab
sub_box = HBox([fth_value, sub_name, sub_h, sub_k, sub_l])
film_tab = VBox([text_film, sub_box])

#Fitting tab
limits = HBox([algo, min_strain, max_strain, min_dw, max_dw])
gsa = HBox([text_gsa,gsa_temp, gsa_cycles, gsa_Tsteps])
#fit_tab = VBox([limits, gsa])
fit_tab = limits


#Generate tabs
tab = widgets.Tab(children=[exp_tab, mater_tab, sdw_tab, film_tab, fit_tab], selected_index = 0)
tab.set_title(0, 'Experiment')
tab.set_title(1, 'Material')
tab.set_title(2, 'Strain / Disorder')
tab.set_title(3, 'Film parameters')
tab.set_title(4, 'Fitting parameters')

#Strain/DW/XRD plots
fig_strain.layout.height = '50%'
fig_strain.layout.width = '640px'
fig_strain.fig_margin=dict(top=30, bottom=15, left=50, right=10)
fig_DW.layout.height = '50%'
fig_DW.layout.width = '640px'
fig_DW.fig_margin=dict(top=0, bottom=40, left=50, right=10)


fig_XRD.layout.width = '640px'
fig_XRD.fig_margin=dict(top=30, bottom=40, left=10, right=0)
fig_XRD.display_toolbar=True

items_layout = Layout(overflow_x = 'visible', overflow_y = 'visible') 
fig_sdw = VBox([fig_strain,fig_DW], layout = items_layout)

fig_tot = HBox([fig_sdw, fig_XRD])

savefitcancel = HBox([fit,save,cancel])

dl_xrd = widgets.HTML("")
dl_sdw = widgets.HTML("")

if local:
    gui = VBox([load_button,tab,fig_tot, savefitcancel,out]) #Local notebook
else:
    gui = VBox([upload_button,tab,fig_tot, savefitcancel, dl_xrd, dl_sdw, out]) #Web-hosted notebook

gui

#**************************
#TODO: implement GSA for offline mode

